## Rao, T., Li, X., & Xu, M. (2020). Learning multi-level deep representations for image emotion classification. Neural processing letters, 51, 2043-2061.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = '/content/drive/My Drive/PROJECT/Image Data/train'
test = '/content/drive/My Drive/PROJECT/Image Data/test'

In [ ]:
# Import necessary libraries and modules
from sklearn.model_selection import train_test_split
import glob
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import os


In [ ]:
train_datagen = ImageDataGenerator(
    width_shift_range = 0.1,        # Randomly shift the width of images by up to 10%
    height_shift_range = 0.1,       # Randomly shift the height of images by up to 10%
    horizontal_flip = True,         # Flip images horizontally at random
    rescale = 1./255,               # Rescale pixel values to be between 0 and 1
    validation_split = 0.2          # Set aside 20% of the data for validation
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255,               # Rescale pixel values to be between 0 and 1
    validation_split = 0.2          # Set aside 20% of the data for validation
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory = train,           # Directory containing the training data
    target_size = (48, 48),          # Resizes all images to 48x48 pixels
    batch_size = 64,                 # Number of images per batch
    color_mode = "grayscale",        # Converts the images to grayscale
    class_mode = "categorical",      # Classifies the images into 7 categories
    subset = "training"              # Uses the training subset of the data
)

validation_generator = validation_datagen.flow_from_directory(
    directory = test,            # Directory containing the validation data
    target_size = (48, 48),          # Resizes all images to 48x48 pixels
    batch_size = 64,                 # Number of images per batch
    color_mode = "grayscale",        # Converts the images to grayscale
    class_mode = "categorical",      # Classifies the images into 7 categories
    subset = "validation"            # Uses the validation subset of the data
)

Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


In [ ]:
# Define a function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    for image_path in image_paths:
        img = image.load_img(image_path, target_size=target_size)
        img = image.img_to_array(img)
        img = preprocess_input(img)
        images.append(img)
    return np.array(images)


# Create labels
safe_labels = np.zeros(len(safe_site_data))  # Label 0 for safe sites
unsafe_labels = np.ones(len(unsafe_site_data))  # Label 1 for unsafe sites

# Combine data and labels
X = np.concatenate((safe_site_data, unsafe_site_data), axis=0)
y = np.concatenate((safe_labels, unsafe_labels), axis=0)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Import necessary Keras modules for building the model
from keras.applications import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.optimizers import Adam

# Load the VGG16 model with pre-trained weights and customize the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Add custom fully connected layers on top of VGG16
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

Epoch 1/10
132/180 ━━━━━━━━━━━━━━━━━━━━ 20:25 26s/step - accuracy: 0.1698 - loss: 2.7376

In [2]:
# Specify training parameters
epochs = 13
batch_size = 32

model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test)
)


Epoch 1/13

23/23 [==============================] - 184s 8s/step - loss: 12.1218 - accuracy: 0.1317 - val_loss: 0.7306 - val_accuracy: 0.1109

Epoch 2/13

23/23 [==============================] - 177s 8s/step - loss: 0.5090 - accuracy: 0.1599 - val_loss: 0.4808 - val_accuracy: 0.2446

Epoch 3/13

23/23 [==============================] - 176s 8s/step - loss: 0.4020 - accuracy: 0.2213 - val_loss: 0.3448 - val_accuracy: 0.2533

Epoch 4/13

23/23 [==============================] - 175s 8s/step - loss: 0.2867 - accuracy: 0.2881 - val_loss: 0.3132 - val_accuracy: 0.1913

Epoch 5/13

23/23 [==============================] - 176s 8s/step - loss: 0.3058 - accuracy: 0.8813 - val_loss: 0.2948 - val_accuracy: 0.8696

Epoch 6/13

23/23 [==============================] - 175s 8s/step - loss: 0.2585 - accuracy: 0.422 - val_loss: 0.2883 - val_accuracy: 0.3076

Epoch 7/13

23/23 [==============================] - 179s 8s/step - loss: 0.2327 - accuracy: 0.5141 - val_loss: 0.3818 - val_accuracy: 0.5022